In [1]:
import os
from hog.hog import train_hog, test_hog, save_hog, load_hog
from utils.load_data import get_test_data, get_train_data

hog_classifier_path = 'models/captcha_classifier_hog.pkl'

if not os.path.exists(hog_classifier_path):
    X_train, y_train = get_train_data()

    classifier = train_hog(X_train, y_train)
    
    save_hog(classifier, hog_classifier_path)
    
else:
    classifier = load_hog(hog_classifier_path)

X_test, y_test = get_test_data()


accuracy = test_hog(classifier, X_test, y_test)

Accuracy: 93.02%
